In [185]:
import pandas as pd
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
from  plotly.subplots  import  make_subplots

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date, datetime, timedelta

In [186]:
#------------------------------------------------------------------------------------PHẦN TIÊU ĐỀ WEB-------------------------------------------------------------------------------------
st.set_page_config(page_icon= 'https://static.wixstatic.com/media/91d4d0_50c2e78106264db2a9ddda29a7ad0503~mv2.png/v1/fit/w_2500,h_1330,al_c/91d4d0_50c2e78106264db2a9ddda29a7ad0503~mv2.png',page_title='THE BIM FACTORY', layout='wide')
st.title('BIM Fee for Raffles MUR TD & SD')

st.sidebar.header("Options filter")

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [187]:
ss_cred_path = r'D:\Documents\Ty\THE BIM FACTORY 4.7.2022\Streamlit_app\credentials2.json' # Your path to the json credential file
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive'] # define the scope
creds = ServiceAccountCredentials.from_json_keyfile_name(ss_cred_path, scope) # add credentials to the account
client = gspread.authorize(creds) # authorize the clientsheet

In [188]:
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1yKc8ZxllaubaL_iMWXGWKHOLZ29_EgXbCg-QGHIO9-Q/edit?usp=sharing')
ws = sheet.worksheet('All data')
df = pd.DataFrame(ws.get_values('A3:L', value_render_option = 'UNFORMATTED_VALUE'))
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df = df.drop(['Notes','Invoice No.', 'Original Amount', 'Currency'], axis=1)
df = df[~df['Client'].str.contains('TBF')]
print(df)

,Status,Client,Project Name,EQ VND (auto),Date Invoice,Date Due,Paid VND,Date Paid
1,0-Fully Paid,PRO-137-ADT,Staffing,138000000,44607,44637,138000000,44649
2,0-Fully Paid,PRO-137-ADT,Staffing,138000000,44635,44665,138000000,44680
3,0-Fully Paid,PRO-137-ADT,Staffing,103500000,44666,44696,103500000,44733
4,0-Fully Paid,PRO-137-ADT,Staffing,103500000,44696,44726,103500000,44748
5,0-Fully Paid,PRO-137-ADT,Staffing,103500000,44727,44757,103500000,44758
...,...,...,...,...,...,...,...,...
351,1-Outstanding,Mr.Minh - (dnb.),Factory Tan Tao Industrial Zone,35900290,,,,44035
352,1-Outstanding,Mr.Minh - (dnb.),Factory Tan Tao - Phase 2,9945000,,,,44036
353,1-Outstanding,Mr.Nam (dnb.),8A Tutoring,29250000,44004,,,
354,1-Outstanding,Fresh Saigon - (dnb.),Juice station - 3D render,6000000,,,,44042


In [189]:
df['Date Invoice']      = df['Date Invoice'].replace("", 0)
df['Date Due']          = df['Date Due'].replace("", 0)
df['Date Paid']         = df['Date Paid'].replace("", 0)
df['Paid VND']          = df['Paid VND'].replace("", 0)


df['Date Invoice']      = df['Date Invoice'].astype(int)
df['Date Due']          = df['Date Due'].astype(int)
df['Date Paid']         = df['Date Paid'].astype(int)

df['EQ VND (auto)']     = df['EQ VND (auto)'].astype(float)
df['Paid VND']          = df['Paid VND'].astype(float)

df['EQ VND (auto)']     = df['EQ VND (auto)'].astype(int)
df['Paid VND']          = df['Paid VND'].astype(int)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 343 entries, 1 to 355
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Status         343 non-null    object
 1   Client         343 non-null    object
 2   Project Name   343 non-null    object
 3   EQ VND (auto)  343 non-null    int32 
 4   Date Invoice   343 non-null    int32 
 5   Date Due       343 non-null    int32 
 6   Paid VND       343 non-null    int32 
 7   Date Paid      343 non-null    int32 
dtypes: int32(5), object(3)
memory usage: 17.4+ KB


In [190]:
def convert_date(number):
    delta = timedelta(days=number)
    start_date = date(1900, 1, 1)
    result_date = start_date + delta
    return result_date.strftime('%m/%d/%Y')


df['Date Invoice']  = df['Date Invoice'].apply(convert_date)
df['Date Due']      = df['Date Due'].apply(convert_date)
df['Date Paid']     = df['Date Paid'].apply(convert_date)

df['Date Invoice']  = pd.to_datetime(df['Date Invoice'])
df['Date Due']      = pd.to_datetime(df['Date Due'])
df['Date Paid']     = pd.to_datetime(df['Date Paid'])
print(df)

,Status,Client,Project Name,EQ VND (auto),Date Invoice,Date Due,Paid VND,Date Paid
1,0-Fully Paid,PRO-137-ADT,Staffing,138000000,2022-02-17,2022-03-19,138000000,2022-03-31
2,0-Fully Paid,PRO-137-ADT,Staffing,138000000,2022-03-17,2022-04-16,138000000,2022-05-01
3,0-Fully Paid,PRO-137-ADT,Staffing,103500000,2022-04-17,2022-05-17,103500000,2022-06-23
4,0-Fully Paid,PRO-137-ADT,Staffing,103500000,2022-05-17,2022-06-16,103500000,2022-07-08
5,0-Fully Paid,PRO-137-ADT,Staffing,103500000,2022-06-17,2022-07-17,103500000,2022-07-18
...,...,...,...,...,...,...,...,...
351,1-Outstanding,Mr.Minh - (dnb.),Factory Tan Tao Industrial Zone,35900290,1900-01-01,1900-01-01,0,2020-07-25
352,1-Outstanding,Mr.Minh - (dnb.),Factory Tan Tao - Phase 2,9945000,1900-01-01,1900-01-01,0,2020-07-26
353,1-Outstanding,Mr.Nam (dnb.),8A Tutoring,29250000,2020-06-24,1900-01-01,0,1900-01-01
354,1-Outstanding,Fresh Saigon - (dnb.),Juice station - 3D render,6000000,1900-01-01,1900-01-01,0,2020-08-01


In [191]:
df['Date Invoice']          = df['Date Invoice'].dt.date
df['Date Due']              = df['Date Due'].dt.date
df['Date Paid']             = df['Date Paid'].dt.date


df = df[df['Date Invoice'] >= date(2022,1,1)]
df

,Status,Client,Project Name,EQ VND (auto),Date Invoice,Date Due,Paid VND,Date Paid
1,0-Fully Paid,PRO-137-ADT,Staffing,138000000,2022-02-17,2022-03-19,138000000,2022-03-31
2,0-Fully Paid,PRO-137-ADT,Staffing,138000000,2022-03-17,2022-04-16,138000000,2022-05-01
3,0-Fully Paid,PRO-137-ADT,Staffing,103500000,2022-04-17,2022-05-17,103500000,2022-06-23
4,0-Fully Paid,PRO-137-ADT,Staffing,103500000,2022-05-17,2022-06-16,103500000,2022-07-08
5,0-Fully Paid,PRO-137-ADT,Staffing,103500000,2022-06-17,2022-07-17,103500000,2022-07-18
...,...,...,...,...,...,...,...,...
278,-1-Disputed,PRO-109-WOAA,Whisky Hotel,34305755,2022-04-25,2022-05-25,0,1900-01-01
279,-1-Disputed,PRO-109-WOAA,Whisky Hotel,137223020,2022-04-25,2022-05-25,0,1900-01-01
286,0-Fully Paid,PRO-140-AXO,2 Technician 10 days,92000000,2022-04-10,2022-04-13,92000000,2022-05-08
287,0-Fully Paid,PRO-140-AXO,2 Technician 10 days,46862500,2022-04-24,2022-04-27,46862500,2022-07-02


In [ ]:
# RUN STREAMLIT FILE
# streamlit run "D:\Documents\Ty\THE BIM FACTORY 4.7.2022\Streamlit_app\TBF_Payments\STL_TBF_Payment.py"

# ---------CREATE REQUIREMENTS FILE
# pip install pipreqs
# pipreqs 'D:\Documents\Ty\THE BIM FACTORY 4.7.2022\Streamlit_app\TBF_Payments' --encoding=utf-8